<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Модули для nlp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
import pickle
import re

In [6]:
train = pd.read_parquet('data/input/data_fusion_train.parquet')
train = train[train.category_id != -1]
print(train.shape)
train.head()

(6483226, 9)


,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,None


In [7]:
type(train)

pandas.core.frame.DataFrame

In [8]:
# Выделение item_name с индексами category_id
train_item_name_ser = train['item_name']
train_item_name_ser.index = train['category_id']
train_item_name_ser = train_item_name_ser.drop_duplicates()
train_item_name_ser

category_id
78                                       Молоко 3,2%,шт
71                            Компот из изюма, 114 ккал
71    Макаронные изделия отварные (масло сливочное),...
70                        Кофе Капучино Большой Эден 18
84                               Хлеб на СЫВОРОТКЕ 350г
                            ...                        
83                       Напиток Энерг. Ред Булл 0,355л
0                                     Хеменгуэй Дайкири
0       Пиво светлое "Халзан" 4,5 % об, пл/б. 1,5 л(шт)
42                                     Экспресс педикюр
84                 Конфеты Харитоша 1кг мол. ваф Яшкино
Name: item_name, Length: 48225, dtype: object

In [9]:
# Предварительная обработка текста
train_item_name_ser = train_item_name_ser.str.lower()
train_item_name_ser

category_id
78                                       молоко 3,2%,шт
71                            компот из изюма, 114 ккал
71    макаронные изделия отварные (масло сливочное),...
70                        кофе капучино большой эден 18
84                               хлеб на сыворотке 350г
                            ...                        
83                       напиток энерг. ред булл 0,355л
0                                     хеменгуэй дайкири
0       пиво светлое "халзан" 4,5 % об, пл/б. 1,5 л(шт)
42                                     экспресс педикюр
84                 конфеты харитоша 1кг мол. ваф яшкино
Name: item_name, Length: 48225, dtype: object

In [10]:
# Приведение в векторное состояние должно получиться < 20 000
stop = stopwords.words('russian')
cv  = CountVectorizer(stop_words = stop, min_df = 3)
cv_fit = cv.fit_transform(train_item_name_ser)
print(f"Определено {cv_fit.shape[1]} оригинальных слов")

Определено 10373 оригинальных слов


In [11]:
word_list = cv.get_feature_names()
count_list = cv_fit.toarray().sum(axis=0)

In [12]:
# Самые популярные слова
pd.Series(dict(zip(word_list,count_list))).sort_values(ascending=False)

шт               2491
5л               1240
1кг              1106
гр                968
хлеб              895
                 ... 
мкр                 3
ганноверские        3
ганча               3
гарнира             3
гигиенические       3
Length: 10373, dtype: int64

In [13]:
X_train = cv_fit
clf = LogisticRegression(max_iter=400)
cross_val_score(clf, X_train, train_item_name_ser.index, cv=3, scoring='f1_weighted')

array([0.77952272, 0.76185511, 0.72258537])

In [14]:
clf.fit(X_train, train_item_name_ser.index)

LogisticRegression(max_iter=400)

In [15]:
predict_list = clf.predict(X_train)
predict_list

array([78, 71, 71, ...,  0, 42, 84], dtype=int64)

In [16]:
predict_list.shape

(48225,)

In [17]:
predict_df = pd.DataFrame(train_item_name_ser)
predict_df['predict'] = predict_list
# Процентная точность модели
print(100 - predict_df.loc[predict_df.index != predict_df.predict].shape[0] / predict_df.shape[0] * 100)
predict_df.loc[predict_df.index != predict_df.predict]

88.42094349403837


,item_name,predict
category_id,,
203,пакет,71
71,морс 200 мл,83
12,400мл mrricco organic п/я67%дп,71
139,"перчатки хоз-ые/кд,s",117
204,участие в программе лояльности,71
...,...,...
111,прокладка кольцо на металлопласт ф16 (10 шт.),105
177,спутниковый ресивер триколор тв,78
103,мормышка литая колюбакино,71


In [18]:
import pickle
tfidf = cv
clf = clf
pickle.dump(tfidf, open('tfidf', 'wb'))
pickle.dump(clf, open('clf_task1', 'wb'))